# NEW DATASET MAKING(2022/10/04)

# Surface Sampling with iner mesh
- I don't know why but XYZ Coordinate is something wrong.
- So I rotate point cloud by `(np.pi/2, 0, 0)`

In [1]:
import open3d as o3d

def pure_surface_sampling(pt_path):
    # 点群ファイルのインポート
    file = pt_path
    # 点群数の定義
    sampling_points = 700000

    # 表面サンプリングと可視化
    mesh = o3d.io.read_triangle_mesh(file)
    
    pcd = o3d.geometry.TriangleMesh.sample_points_uniformly(mesh,number_of_points=sampling_points)

    return pcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import copy
import numpy as np

# Rotate point cloud
def rotation(pcd):
    pcd_c = copy.deepcopy(pcd)
    R = pcd.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
    pcd_c.rotate(R, center=(0,0,0))
    return pcd_c

In [5]:
from glob import glob
import os 

output_dir = '/home/takenobu/ダウンロード/new_obj_files/edit/pointcloud/Surface_with_Iner/'
source_dir = '/home/takenobu/ダウンロード/new_obj_files/edit/obj/*.obj'

data_list = glob(source_dir)

for data in data_list:
    try:
        basename = os.path.basename(data)
        pcd = pure_surface_sampling(data)
        pcd_r = rotation(pcd)
        core_name = basename[:-4] + '.ply'
        save_name = output_dir + core_name

        o3d.io.write_point_cloud(save_name, pcd_r)
        
    except:
        basename = os.path.basename(data)
        core_name = basename[:-4] + '.ply'
        save_name = output_dir + core_name
        print('=========')
        print('ERORR : ', save_name)
        print('=========')
        continue

    

[Open3D INFO] Skipping non-triangle primitive geometry of type: 6
ERORR :  /home/takenobu/ダウンロード/new_obj_files/edit/pointcloud/Surface_with_Iner/34.ply
[Open3D INFO] Skipping non-triangle primitive geometry of type: 6
[Open3D INFO] Skipping non-triangle primitive geometry of type: 6
[Open3D INFO] Skipping non-triangle primitive geometry of type: 6
[Open3D INFO] Skipping non-triangle primitive geometry of type: 6


### 34.plyで異常発生 

In [6]:
for i in range(61):
    surface_with_iner_DIR = '/home/takenobu/ダウンロード/new_obj_files/edit/pointcloud/Surface_with_Iner/{}.ply'.format(i+1)
    if os.path.exists(surface_with_iner_DIR):
        continue
    else:
        print(i+1, 'is lost by some resones..')

9 is lost by some resones..
13 is lost by some resones..
31 is lost by some resones..
34 is lost by some resones..
35 is lost by some resones..
38 is lost by some resones..
53 is lost by some resones..
58 is lost by some resones..


In [15]:
61-8

53

# Directly Visibility PointSets

In [16]:
import numpy as np
import open3d as o3d
import math
import random
import os
from tqdm import tqdm
import glob

def surface_sampling(pt_path):
    # 点群ファイルのインポート
    file = pt_path
    # 点群数の定義
    sampling_points = 700000

    #ダウンサンプリングの点群数
    result_points = 8192
    
    pcd = o3d.io.read_point_cloud(file)

    # 表面サンプリングと可視化
    print('=====================')
    print(pcd)
    print('=====================')

    # なんかようわからんが、この計算で点群を球で囲ったときの直径が取れるらしい.....
    diameter = np.linalg.norm(np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))

    # 逆投影するための外円定義 and この変数自体は(by　武田修論)
    
    ## ========================================================================
    radius = diameter * 29
     ## ========================================================================

    # 見えた点群のインデックスをここに記録
    visible_index = []
    # 貼る格子の１辺のカメラ個数
    lattice_number = 8

    ###directly visibility of pointsets###

    side = int(1.3 * diameter)

    ## side1(+側でx固定)
    # first_location
    first_location_x = int(side/2) 
    first_location_y = int(side/2) * (-1)
    first_location_z = side

    for i in range(lattice_number):
        # z軸決定
        location_z = first_location_z - i*(side/(lattice_number-1))
        for j in range(lattice_number):
            # y軸決定
            location_y = first_location_y + j*(side/(lattice_number-1))
            
            camera = [first_location_x, location_y, location_z]
            _, pt_map = pcd.hidden_point_removal(camera, radius)
            visible_index.extend(pt_map)
            

    # side2(+側でy固定)
    # first_location
    first_location_x = int(side/2)
    first_location_y = int(side/2) 
    first_location_z = side

    for i in range(lattice_number):
        # z軸決定
        location_z = first_location_z - i*(side/(lattice_number-1))
        for j in range(lattice_number):
            # x軸決定
            location_x = first_location_x - j*(side/(lattice_number-1))
            camera = [location_x, first_location_y, location_z]
            _, pt_map = pcd.hidden_point_removal(camera, radius)
            visible_index.extend(pt_map)

    # side3(-側でx固定)
    first_location_x = int(side/2) * (-1)
    first_location_y = int(side/2) * (-1)
    first_location_z = side

    for i in range(lattice_number):
        # z軸決定
        location_z = first_location_z - i*(side/(lattice_number-1))
        for j in range(lattice_number):
            # y軸決定
            location_y = first_location_y + j*(side/(lattice_number-1))
            camera = [first_location_x, location_y, location_z]
            _, pt_map = pcd.hidden_point_removal(camera, radius)
            visible_index.extend(pt_map)

    # side4(-側でy固定)
    first_location_x = int(side/2) * (-1)
    first_location_y = int(side/2) * (-1)
    first_location_z = side

    for i in range(lattice_number):
        # z軸決定
        location_z = first_location_z - i*(side/(lattice_number-1))
        for j in range(lattice_number):
            # x軸決定
            location_x = first_location_x + j*(side/(lattice_number-1))
            camera = [location_x, first_location_y, location_z]
            _, pt_map = pcd.hidden_point_removal(camera, radius)
            visible_index.extend(pt_map)

    # top(z軸で固定)
    first_location_x = int(side/2)
    first_location_y = int(side/2)
    first_location_z = side

    for i in range(lattice_number):
        # z軸決定
        location_y = first_location_y - i*(side/(lattice_number-1))
        for j in range(lattice_number):
            # x軸決定
            location_x = first_location_x - j*(side/(lattice_number-1))
            camera = [location_x, location_y, first_location_z]
            _, pt_map = pcd.hidden_point_removal(camera, radius)
            visible_index.extend(pt_map)

    # set型→list型に変換して重複削除・並び替え
    visible_index = list(set(visible_index))

    # ランダムサンプリングの処理
    result_index = random.sample(visible_index, result_points)

    # 表面サンプリング＆ダウンサンプリング
    pcd_downsample = pcd.select_by_index(result_index)
    # 表面サンプリング
    pcd_surface = pcd.select_by_index(visible_index)

    return pcd_surface, pcd_downsample


In [17]:
from glob import glob
import open3d as o3d
from tqdm import tqdm

source_dir = '/home/workspace/kiyama/PoinTr/new_obj_files/edit/pointcloud/Surface_with_Iner/'
output_dir = '/home/workspace/kiyama/PoinTr/new_obj_files/edit/pointcloud/Surface_without_iner/'
downsample_dir = '/home/workspace/kiyama/PoinTr/new_obj_files/edit/pointcloud/Surface_without_Iner_downsample/'

ply_data_path_list = glob(source_dir + '*.ply')

for data_path in tqdm(ply_data_path_list):
    # basename を取得
    basename = os.path.basename(data_path)
    outputname = basename[:-4]+'.ply'
    try:
        # 表面サンプリングの実行
        pcd_surface, pcd_downsample = surface_sampling(data_path)
        
        print('=====================')
        print('BASE NAME : ' + basename)
        print('pdc_surface : ')
        print(pcd_surface)
        print('pdc_downsample : ')
        print(pcd_downsample)
        print('=====================')

        #  点群データの保存
        o3d.io.write_point_cloud(output_dir+outputname, pcd_surface)
        o3d.io.write_point_cloud(downsample_dir+outputname, pcd_downsample)
    
    except:
        print('=====================')
        print(basename, ' で問題が起きたっぽい')
        print('=====================')
        continue

  0%|          | 0/53 [00:00<?, ?it/s]

PointCloud with 700000 points.


  2%|▏         | 1/53 [01:37<1:24:29, 97.50s/it]

BASE NAME : 33.ply
pdc_surface : 
PointCloud with 141505 points.
pdc_downsample : 
PointCloud with 8192 points.
PointCloud with 700000 points.


  4%|▍         | 2/53 [03:33<1:32:13, 108.49s/it]

BASE NAME : 18.ply
pdc_surface : 
PointCloud with 217917 points.
pdc_downsample : 
PointCloud with 8192 points.
PointCloud with 700000 points.
